In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
week_number = 7

In [ ]:
plays = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL/plays.csv',None)

<ipython-input-5-edf91440e766>:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  plays = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL/plays.csv',None)
<ipython-input-5-edf91440e766>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  plays = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL/plays.csv',None)


In [ ]:
week_df = pd.read_csv(f'/content/drive/MyDrive/NFL DATA BOWL/tracking_week_{week_number}.csv',None)

<ipython-input-6-6dc3bfa3b0d6>:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  week_df = pd.read_csv(f'/content/drive/MyDrive/NFL DATA BOWL/tracking_week_{week_number}.csv',None)
<ipython-input-6-6dc3bfa3b0d6>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  week_df = pd.read_csv(f'/content/drive/MyDrive/NFL DATA BOWL/tracking_week_{week_number}.csv',None)


In [ ]:

# Extract 'los_x' from 'week_df' where 'displayName' is 'football' and 'frameId' is 1
los_x = week_df[(week_df['displayName'] == 'football') & (week_df['frameId'] == 1)][['gameId', 'playId', 'x']].rename(columns={'x': 'los_x'})

# Merge 'los_x' back into 'week_df'
week_df = week_df.merge(los_x, on=['gameId', 'playId'], how='left')


In [ ]:
# Merging week_df with plays_df
result_df = week_df.merge(plays, on=['gameId', 'playId'], how='left')


In [ ]:
#MERGED_DF IS JUST AN EXAMPLE

result_df['ToLeft'] = 0
result_df['ToLeft'] = result_df['playDirection'].apply(lambda x: 1 if x == 'left' else 0)

result_df['YardsFromOwnGoal'] = result_df[['yardlineSide','possessionTeam','yardlineNumber']].apply(lambda row:row.yardlineNumber if row.yardlineSide==row.possessionTeam else 50+(50-row.yardlineNumber), axis=1)

result_df['YardsFromOwnGoal'] = result_df[['YardsFromOwnGoal','yardlineNumber']].apply(lambda row:50 if row.yardlineNumber==50 else row.YardsFromOwnGoal, axis=1)

result_df['X_std'] = result_df[['ToLeft','x']].apply(lambda row:120-row.x-10 if row.ToLeft else row.x-10, axis=1)

result_df['Y_std'] = result_df[['ToLeft','y']].apply(lambda row:160/3-row.y if row.ToLeft else row.y, axis=1)

result_df['Dir_std_1'] = result_df[['ToLeft','dir']].apply(lambda row:row.dir+360 if (row.ToLeft and row.dir<90) else row.dir, axis=1)
result_df['Dir_std_1'] = result_df[['ToLeft','dir','Dir_std_1']].apply(lambda row:row.dir-360 if ((not row.ToLeft) and row.dir>270) else row.Dir_std_1, axis=1)
result_df['Dir_std_2'] = result_df[['ToLeft','Dir_std_1']].apply(lambda row:row.Dir_std_1-180 if row.ToLeft else row.Dir_std_1, axis=1)

import numpy as np

# Assuming 'displayName' and 'ballCarrierDisplayName' are columns in result_df
result_df['ballcarrier'] = np.where(result_df['displayName'] == result_df['ballCarrierDisplayName'], 1, 0)


KeyboardInterrupt: ignored

In [ ]:
position = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL/players.csv')

In [ ]:
result_df = result_df.merge(position, on=['nflId'], how='left')
result_df = result_df.rename(columns={'displayName_x': 'displayName'})


In [ ]:
result_df.head()

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event,los_x,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,penaltyYards,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,offenseFormation,defendersInTheBox,passProbability,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2,ToLeft,YardsFromOwnGoal,X_std,Y_std,Dir_std_1,Dir_std_2,ballcarrier,height,weight,birthDate,collegeName,position,displayName_y,Qualifies
0,2022102000,56,37084.0,J.J. Watt,1,2022-10-20 20:16:19.099999,99.0,ARI,left,87.64,23.49,3.95,2.68,0.40,113.01,67.50,NaN,85.669998,52942,Juwan Johnson,(15:00) (Shotgun) A.Dalton pass short middle t...,1,1,10,NO,ARI,NO,25,15:00,0,0,C,5.0,NaN,6,6,N,85,SHOTGUN,7.0,0.497866,0.554282,0.445718,0.00049,-0.00049,1.298699,0.027666,NaN,NaN,NaN,NaN,1,25,22.36,29.843333,427.50,247.50,0,6-5,288.0,1989-03-22,Wisconsin,DE,J.J. Watt,NaN
1,2022102000,56,37084.0,J.J. Watt,2,2022-10-20 20:16:19.200000,99.0,ARI,left,88.02,23.63,4.08,2.35,0.41,118.70,71.05,pass_arrived,85.669998,52942,Juwan Johnson,(15:00) (Shotgun) A.Dalton pass short middle t...,1,1,10,NO,ARI,NO,25,15:00,0,0,C,5.0,NaN,6,6,N,85,SHOTGUN,7.0,0.497866,0.554282,0.445718,0.00049,-0.00049,1.298699,0.027666,NaN,NaN,NaN,NaN,1,25,21.98,29.703333,431.05,251.05,0,6-5,288.0,1989-03-22,Wisconsin,DE,J.J. Watt,NaN
2,2022102000,56,37084.0,J.J. Watt,3,2022-10-20 20:16:19.299999,99.0,ARI,left,88.44,23.74,4.21,2.07,0.44,114.82,75.53,NaN,85.669998,52942,Juwan Johnson,(15:00) (Shotgun) A.Dalton pass short middle t...,1,1,10,NO,ARI,NO,25,15:00,0,0,C,5.0,NaN,6,6,N,85,SHOTGUN,7.0,0.497866,0.554282,0.445718,0.00049,-0.00049,1.298699,0.027666,NaN,NaN,NaN,NaN,1,25,21.56,29.593333,435.53,255.53,0,6-5,288.0,1989-03-22,Wisconsin,DE,J.J. Watt,NaN
3,2022102000,56,37084.0,J.J. Watt,4,2022-10-20 20:16:19.400000,99.0,ARI,left,88.86,23.82,4.20,2.07,0.43,121.02,79.59,NaN,85.669998,52942,Juwan Johnson,(15:00) (Shotgun) A.Dalton pass short middle t...,1,1,10,NO,ARI,NO,25,15:00,0,0,C,5.0,NaN,6,6,N,85,SHOTGUN,7.0,0.497866,0.554282,0.445718,0.00049,-0.00049,1.298699,0.027666,NaN,NaN,NaN,NaN,1,25,21.14,29.513333,439.59,259.59,0,6-5,288.0,1989-03-22,Wisconsin,DE,J.J. Watt,NaN
4,2022102000,56,37084.0,J.J. Watt,5,2022-10-20 20:16:19.500000,99.0,ARI,left,89.28,23.88,4.15,2.08,0.42,124.76,82.73,NaN,85.669998,52942,Juwan Johnson,(15:00) (Shotgun) A.Dalton pass short middle t...,1,1,10,NO,ARI,NO,25,15:00,0,0,C,5.0,NaN,6,6,N,85,SHOTGUN,7.0,0.497866,0.554282,0.445718,0.00049,-0.00049,1.298699,0.027666,NaN,NaN,NaN,NaN,1,25,20.72,29.453333,442.73,262.73,0,6-5,288.0,1989-03-22,Wisconsin,DE,J.J. Watt,NaN


In [ ]:
#play_check = result_df[result_df['gameId']==2022090800]
#specific = play_check[play_check['playId']==2072]

In [ ]:
import pandas as pd
import numpy as np

# Your existing function (slightly modified to return the 'qualifies' value)
def does_run_qualify(tracking_for_play):
    qualifies = 0

    filtered_df = tracking_for_play[(tracking_for_play['ballcarrier'] == 1) & (tracking_for_play['frameId'] == 1)]
    if not filtered_df.empty:
        offense_club = filtered_df['club'].iloc[0]
    else:
        print("NO TEAM GIVEN")
        return

    los = tracking_for_play[(tracking_for_play['displayName'] == 'football') & (tracking_for_play['frameId'] == 1)]
    los_x = los['x'].iloc[0]

    behind_los_frames = []
    for index, row in tracking_for_play.iterrows():
        if row['playDirection'] == 'right' and row['displayName'] == 'football':
            if row['x'] < los_x:
                behind_los_frames.append(row['frameId'])
        elif row['playDirection'] == 'left' and row['displayName'] == 'football':
            if row['x'] > los_x:
                behind_los_frames.append(row['frameId'])

    try:
        last_frame_behind_los = max(behind_los_frames)
    except ValueError:
        print("ODDITY", row['playId'])
        return

    behind_los_tracking = tracking_for_play[tracking_for_play['frameId'] <= last_frame_behind_los]

    tackles = behind_los_tracking[(behind_los_tracking['position'] == 'T') & (behind_los_tracking['frameId'] == 1)]
    tackles_y = tackles['y'].to_list()

    if len(tackles_y) == 1:
        guards = behind_los_tracking[(behind_los_tracking['position'] == 'G') & (behind_los_tracking['frameId'] == 1)]
        guards_y = guards['y'].to_list()
        tackles_y.append(guards_y)
        tackles_y = [item for sublist in tackles_y for item in (sublist if isinstance(sublist, list) else [sublist])]

    for index, row in behind_los_tracking.iterrows():
        if row['ballcarrier'] == 1:
            x, y, direction, speed = row["x"], row["y"], row["dir"], row["s"]
            adjusted_direction = np.mod(90 - direction, 360)
            x_comp = np.cos(np.radians(adjusted_direction))
            y_comp = np.sin(np.radians(adjusted_direction))
            v_x = speed * x_comp
            v_y = speed * y_comp
            mu_val_x = x + v_x * 0.5
            mu_val_y = y + v_y * 0.5
            mu = [mu_val_x, mu_val_y]
            min_tackle_value = min(tackles_y)
            max_tackle_value = max(tackles_y)
            is_between = min_tackle_value <= mu_val_y <= max_tackle_value

            if not is_between:
                qualifies = 1
                break

    return qualifies

# New function to iterate over games and plays
def check_all_runs(results_df):
    results = []

    for game_id in results_df['gameId'].unique():
        game_data = results_df[results_df['gameId'] == game_id]

        for play_id in game_data['playId'].unique():
            play_data = game_data[game_data['playId'] == play_id]
            qualifies = does_run_qualify(play_data)
            results.append({'gameId': game_id, 'playId': play_id, 'Qualifies': qualifies})

    return pd.DataFrame(results)

# Example usage (assuming results_df is your DataFrame)
final_results_df = check_all_runs(result_df)



In [ ]:
result_df = result_df.merge(final_results_df, on=['gameId', 'playId'])
qualified = result_df[result_df['Qualifies']==1]

In [ ]:
def left_or_right_run_intention(week7_tracking_runs_qualified, game_id, play_id):

    tracking_for_play = week7_tracking_runs_qualified[(week7_tracking_runs_qualified['gameId'] == game_id) & (week7_tracking_runs_qualified['playId'] == play_id)]

    threshold_tracking = tracking_for_play[(tracking_for_play['frameId'] <= 25) & (tracking_for_play['ballcarrier'] == 1)]

    ballcarrier_dirs = []

    for index, row in threshold_tracking.iterrows():
        ballcarrier_dirs.append(row['Dir_std_2'])

    left_count = sum(0 <= angle <= 90 for angle in ballcarrier_dirs)
    right_count = sum(90 < angle <= 180 for angle in ballcarrier_dirs)

    intended_direction = ""

    if left_count > right_count:
        intended_direction += "left"
    elif right_count > left_count:
        intended_direction += "right"

    return intended_direction


from collections import Counter
def edge_setter_in_question(runs_that_qualify, game_id, play_id):

    tracking_for_play = runs_that_qualify[(runs_that_qualify['gameId'] == game_id) & (runs_that_qualify['playId'] == play_id)]

    los = tracking_for_play[(tracking_for_play['displayName'] == 'football') & (tracking_for_play['frameId'] == 1)]
    los_x = los['X_std'].iloc[0]

    positions = ["CB", "DB", "DE", "OLB"]

    ballcarrier_tracking_frames = tracking_for_play[tracking_for_play['ballcarrier'] == 1]
    target_tracking = tracking_for_play[(tracking_for_play['position'].isin(positions)) | (tracking_for_play['ballcarrier'] == 1)]

    ### GIVES US LOS FRAME
    mask = ballcarrier_tracking_frames['X_std'] > los_x

    first_valid_index = ballcarrier_tracking_frames[mask].first_valid_index()

    end_frame = ballcarrier_tracking_frames.loc[first_valid_index, 'frameId'] if first_valid_index is not None else None

    if end_frame is None:
        first_contact_index = ballcarrier_tracking_frames[ballcarrier_tracking_frames['event'] == 'first_contact'].first_valid_index()
        end_frame = ballcarrier_tracking_frames.loc[first_contact_index, 'frameId'] if first_contact_index is not None else ballcarrier_tracking_frames['frameId'].max()

    target_tracking = target_tracking[target_tracking['frameId'] <= end_frame]
    ballcarrier_tracking_filtered = ballcarrier_tracking_frames[ballcarrier_tracking_frames['frameId'] <= end_frame]

    ballcarrier_data = ballcarrier_tracking_filtered.copy()
    ballcarrier_data['x_comp'] = np.cos(np.radians(ballcarrier_data['Dir_std_2']))
    ballcarrier_data['y_comp'] = np.sin(np.radians(ballcarrier_data['Dir_std_2']))
    ballcarrier_data['v_x'] = ballcarrier_data['s'] * ballcarrier_data['x_comp']
    ballcarrier_data['v_y'] = ballcarrier_data['s'] * ballcarrier_data['y_comp']
    ballcarrier_data['mu_val_x'] = ballcarrier_data['X_std'] + ballcarrier_data['v_x'] * 0.5
    ballcarrier_data['mu_val_y'] = ballcarrier_data['Y_std'] + ballcarrier_data['v_y'] * 0.5

    closest_defenders = {}

    for frame_id in ballcarrier_data['frameId'].unique():
        # Ball carrier's target point for this frame
        bx = ballcarrier_data.loc[ballcarrier_data['frameId'] == frame_id, 'mu_val_x'].iloc[0]
        by = ballcarrier_data.loc[ballcarrier_data['frameId'] == frame_id, 'mu_val_y'].iloc[0]

        # Filter for defenders in the same frame
        frame_defenders = target_tracking[(target_tracking['frameId'] == frame_id) & (target_tracking['ballcarrier'] != 1)].copy()

        # Vectorized distance calculation
        frame_defenders['distance_to_target'] = np.sqrt((frame_defenders['X_std'] - bx) ** 2 + (frame_defenders['Y_std'] - by) ** 2)

        if not frame_defenders.empty:
            closest_defender = frame_defenders.loc[frame_defenders['distance_to_target'].idxmin()]
            closest_defenders[frame_id] = closest_defender['displayName']

        else:
            closest_defenders[frame_id] = None

    name_counter = Counter(closest_defenders.values())

    edge_rusher = name_counter.most_common(1)[0][0]
    return edge_rusher

In [ ]:
def analyze_runs_in_qualified(qualified):
    results = []

    for game_id in qualified['gameId'].unique():
        game_data = qualified[qualified['gameId'] == game_id]

        for play_id in game_data['playId'].unique():
            play_data = game_data[game_data['playId'] == play_id]

            # Call your functions for each game and play
            run_intention = left_or_right_run_intention(qualified, game_id, play_id)
            edge_rusher = edge_setter_in_question(qualified, game_id, play_id)

            # Store the results
            results.append({'gameId': game_id, 'playId': play_id, 'RunIntention': run_intention, 'EdgeRusher': edge_rusher})

    return pd.DataFrame(results)

# Example usage
final_analysis_df = analyze_runs_in_qualified(qualified)



In [ ]:

qualified = qualified.merge(final_analysis_df, on=['gameId', 'playId'])


In [ ]:
qualified = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL checkpoints/qualified_runs_week_7.csv')

In [ ]:
import matplotlib.patches as patches
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.patches import Arrow

# FUNCTION TO CREATE FOOTBALL FIELD

def create_football_field():

    # Create a rectangle defined via an anchor point *xy* and its *width* and *height*
    rect = patches.Rectangle((0, 0), 120, 53.3, facecolor='green', alpha = 0.7, zorder=0)

    # Creating a subplot to plot our field on
    fig, ax = plt.subplots(1, figsize=(12, 6.33))

    # Adding the rectangle to the plot
    ax.add_patch(rect)

    # Plotting a line plot for marking the field lines
    #plt.plot([10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              #80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             #[0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3,
             # 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             #color='white', zorder = 0)

    # Creating the left end-zone
    left_end_zone = patches.Rectangle((0, 0), 10, 53.3, facecolor='blue', alpha=0.2, zorder=0)

    # Creating the right end-zone
    right_end_zone = patches.Rectangle((110, 0), 120, 53.3, facecolor='blue', alpha=0.2, zorder=0)

    # Adding the patches to the subplot
    ax.add_patch(left_end_zone)
    ax.add_patch(right_end_zone)

    # Setting the limits of x-axis from 0 to 120
    #plt.xlim(0, 120)

    # Setting the limits of y-axis from -5 to 58.3
    #plt.ylim(-5, 58.3)

    # Removing the axis values from the plot
    #plt.axis('off')

    # Plotting the numbers starting from x = 20 and ending at x = 110
    # with a step of 10
    for x in range(20, 110, 10):

        # Intializing another variable named 'number'
        number = x

        # If x exceeds 50, subtract it from 120
        if x > 50:
            number = 120 - x

        # Plotting the text at the bottom
        plt.text(x, 5, str(number - 10),
                 horizontalalignment='center',
                 fontsize=20,
                 color='white')

        # Plotting the text at the top
        plt.text(x - 0.95, 53.3 - 5, str(number - 10),
                 horizontalalignment='center',
                 fontsize=20,
                 color='white',
                 rotation=180)

    # Making ground markings
    for x in range(11, 110):
            ax.plot([x, x], [0.4, 0.7], color='white', zorder = 0)
            ax.plot([x, x], [53.0, 52.5], color='white', zorder = 0)
            ax.plot([x, x], [22.91, 23.57], color='white', zorder = 0)
            ax.plot([x, x], [29.73, 30.39], color='white', zorder = 0)

    # Returning the figure and axis
    return fig, ax


def calculate_voronoi_regions(player_coordinates):
    vor = Voronoi(player_coordinates)

    voronoi_regions = []

    for r in range(len(player_coordinates)):
        region = vor.regions[vor.point_region[r]]
        if not -1 in region:
            polygon_coords = [vor.vertices[i] for i in region]
            voronoi_regions.append(polygon_coords)

    return voronoi_regions

In [ ]:
tracking = qualified.copy()
ballcarrier = tracking[tracking['ballcarrier']==1]
ballcarrier = ballcarrier.rename(columns={col: f'ballcarrier_{col}' for col in ballcarrier.columns})
# Create a subset of 'ballcarrier' with only the necessary columns
ballcarrier_subset = ballcarrier[['ballcarrier_gameId', 'ballcarrier_playId','ballcarrier_frameId', 'ballcarrier_s', 'ballcarrier_x']]




In [ ]:

# Optionally, you can drop the redundant columns after merging if you want
tracking = tracking.merge(ballcarrier_subset, left_on=['gameId', 'playId', 'frameId'], right_on=['ballcarrier_gameId', 'ballcarrier_playId', 'ballcarrier_frameId'])



In [ ]:
tracking['speed_metric'] = (tracking['ballcarrier_s'] > 0).astype(int)

# Sort DataFrame by playId and then by frameId
tracking = tracking.sort_values(by=['playId', 'frameId'])

# Identify where speed_metric goes from 1 to 0
tracking['speed_drops_to_zero'] = (tracking['speed_metric'] == 0) & (tracking['speed_metric'].shift(1) == 1)

# Group by playId and find the first occurrence of speed dropping to 0 in each play
first_drop_indices = tracking.groupby('playId')['speed_drops_to_zero'].transform('idxmax')

# Create a mask to keep rows after the first drop to zero in each play
mask = tracking.index >= first_drop_indices

# Apply the mask to filter the DataFrame
tracking = tracking[mask]


In [ ]:
import numpy as np
# Calculate if the ballcarrier is past the threshold in each frame
tracking['is_past_threshold'] = np.where(
    ((tracking['playDirection'] == 'right') & (tracking['ballcarrier_x'] > tracking['los_x'])) |
    ((tracking['playDirection'] == 'left') & (tracking['ballcarrier_x'] < tracking['los_x'])),
    1, 0
)

# Use cummax to ensure that once the threshold is passed, it remains passed for all subsequent frames
tracking['past_threshold'] = tracking.groupby(['gameId', 'playId'])['is_past_threshold'].cummax()



In [ ]:
tracking= tracking[tracking['past_threshold']==0]

In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from sklearn.cluster import KMeans
from collections import Counter
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.path as mplPath
from shapely.geometry import Polygon as PolygonArea
import matplotlib as mpl
import random
import os
from itertools import chain
from matplotlib.patches import Circle
from scipy.spatial import Voronoi,voronoi_plot_2d, ConvexHull
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.path as mplPath
from shapely.geometry import Polygon as PolygonArea
cmap = mpl.colormaps['viridis']

data_for_dataframe = []

# EXAMPLE FUNCTION USE AT BOTTOM, PLUG IN GAME_ID, PLAY_ID, & TRACKING (WEEK7_TRACKING WAS ALL TRACKING FOR WEEK7)

#for frame_id in range(7, 44):
    #calculate_overlap_for_frame(frame_id, 2022102309, 3809, week7_tracking, cumulative_overlap_dict)

# EXAMPLE USE ^ — 7 AND 44 ARE THE STARTING & ENDING FRAMES
# STARTING FRAME FOR ALL PLAYS IS WHEN BALLCARRIER'S SPEED > 0

# ENDING FRAME FOR ALL OUTSIDE RUNS IS WHEN THE BALLCARRIER CROSSES THE LOS
# NEED TO FIGURE OUT ENDING FRAME FOR NON-OUTSIDE RUNS (LIKELY WHEN EDGE RUSHER'S TARGET == 0)
# NEED TO FIGURE OUT HOW TO CLASSIFY A RUN AS EDGE-SETTING WORTHY (LIKELY WHEN VECTOR REACHES OUTSIDE THE TACKLES?)
cumulative_overlap_dict = {}

# INSERT PLAY_ID & GAME_ID (FOR WEEK 7 AT LEAST)

def calculate_overlap_for_frame(frame_id, game_id, play_id, tracking, cumulative_dict):

    tracking_all = tracking[(tracking['playId'] == play_id)
                                 & (tracking['gameId'] == game_id)
                                 & (tracking['frameId'] == frame_id)]

    tracking_all = tracking_all.sort_values(by='club')
    tracking_all_no_football = tracking_all[tracking_all['club'] != 'football']

    # FOR GRAIENT COLORING ON PROB. DISTRIBUTION FUNCTION

    x, y = np.mgrid[0:120:1, 0:53.3:1]
    locations = np.dstack((x, y))
    x, y = np.mgrid[0:120:1, 0:53.3:1]
    fig, ax = create_football_field()


    ballcarrier = tracking_all[tracking_all['ballcarrier'] == 1]
    offensive_team = ballcarrier['club'].to_list()[0]



    offensive_data = tracking_all[(tracking_all['frameId'] == frame_id) & (tracking_all['club'] == offensive_team)]
    defense_data = tracking_all[(tracking_all['frameId'] == frame_id) & (tracking_all['club'] != offensive_team) & (tracking_all['club'] != 'football')]
    football_data = tracking_all[(tracking_all['frameId'] == frame_id) & (tracking_all['club'] == 'football')]

    offensive_x, offensive_y = offensive_data['x'], offensive_data['y']
    defense_x, defense_y = defense_data['x'], defense_data['y']
    football_x, football_y = football_data['x'], football_data['y']

    xy = tracking_all_no_football[['x','y']].values

    n_points = xy.shape[0]
    xy1 = xy.copy()
    xy1[:,1] = - xy[:,1]
    xy2 = xy.copy()
    xy2[:,1] = 320/3 - xy[:,1]
    xy3 = xy.copy()
    xy3[:,0] = 30 - xy[:,0]
    xy4 = xy.copy()
    xy4[:,0] = 220 - xy[:,0]
    xy = np.concatenate((xy, xy1, xy2, xy3, xy4), axis=0)

    voronoi_regions1 = calculate_voronoi_regions(xy)

    num_regions = len(voronoi_regions1)
    colors = ['red'] * 11 + ['lightblue'] * (11)

    polygons = [Polygon(region) for region in voronoi_regions1]

    plot = PatchCollection(polygons, facecolors = colors, alpha=0.35)

    ax.set_aspect('equal')

    for index, row in ballcarrier.iterrows():

        x, y, direction, speed = row["x"], row["y"], row["dir"], row["s"]
        ballcarrier_x = row['x']
        ballcarrier_y = row['y']

        speed_ratio = (speed**2)/(100)

        adjusted_direction = np.mod(90 - direction, 360)  # Apply the adjustment

        x_comp = np.cos(np.radians(adjusted_direction))
        y_comp = np.sin(np.radians(adjusted_direction))

        v_x = speed * x_comp
        v_y = speed * y_comp

        arrow = Arrow(x, y, v_x, v_y, width=1, color='white', zorder=2)

        scaling_matrix = np.diag([abs(v_x), abs(v_y)])

        if 215 < adjusted_direction < 315 or 45 < adjusted_direction < 135:

            rotation_matrix = np.array([[np.cos(np.radians(adjusted_direction - 90)), -np.sin(np.radians(adjusted_direction - 90))],
                                       [np.sin(np.radians(adjusted_direction - 90)), np.cos(np.radians(adjusted_direction - 90))]])

            rotation_matrix_T = rotation_matrix.T

            covariance_matrix = rotation_matrix @ scaling_matrix @ rotation_matrix_T

        else:

            rotation_matrix = np.array([[np.cos(np.radians(adjusted_direction)), -np.sin(np.radians(adjusted_direction))],
                                       [np.sin(np.radians(adjusted_direction)), np.cos(np.radians(adjusted_direction))]])

            rotation_matrix_T = rotation_matrix.T

            covariance_matrix = rotation_matrix @ scaling_matrix @ rotation_matrix_T
            covariance_matrix += np.eye(covariance_matrix.shape[0]) * 1e-8

        mu_val_x = x + v_x * 0.5
        mu_val_y = y + v_y * 0.5
        mu = [mu_val_x,mu_val_y]

        player_pdf = multivariate_normal(mu,covariance_matrix).pdf(locations)

        player_pdf_df = pd.DataFrame(data=player_pdf)

        player_pdf_values = player_pdf_df.values
        new_cmap = LinearSegmentedColormap.from_list('custom_cmap', ['green', 'green'])


        #normalized_player_pdf_values = (player_pdf_values - np.min(player_pdf_values)) / (np.max(player_pdf_values) - np.min(player_pdf_values))

        x, y = np.mgrid[0:120:1, 0:53.3:1]
        exact_contour_filled = ax.contourf(x, y, player_pdf_values, cmap=new_cmap, levels=1, zorder=2)
        viz_contour_filled = ax.contourf(x, y, player_pdf_values, cmap=new_cmap, levels=25, zorder=2)


        #football_scatter = ax.scatter(football_x, football_y, c='brown', edgecolors='white', alpha=0.75, linewidth=0.75, s=40, zorder=3)
        voronoi_player_dict = {}

        n_players = min(len(tracking_all_no_football['displayName']), 22)
        n_regions = min(len(voronoi_regions1), 22)

        for player, region in zip(tracking_all_no_football['displayName'][:n_players], voronoi_regions1[:n_regions]):
            voronoi_player_dict[player] = region

        defense_players_list = defense_data['displayName'].to_list()

        voronoi_player_dict_defense = {}

        for key, value in voronoi_player_dict.items():
            if key in defense_players_list:
                voronoi_player_dict_defense[key] = value

        voronoi_player_dict_defense_clean = {player: [array.tolist() for array in regions]
                                         for player, regions in voronoi_player_dict_defense.items()}

        values = voronoi_player_dict_defense_clean.values()
        list_vals = list(values)

        for verts in list_vals:
            for cord in verts:
                x, y = cord
                ax.scatter(x, y, c='white', s=10)


        contour_path = exact_contour_filled.collections[0].get_paths()[0]
        countour_vertices = contour_path.vertices
        countour_vertices_list = countour_vertices.tolist()

        countour_cleaned_vertices = []

        for pair in countour_vertices_list:
            if len(str(pair[0])) > 5 or len(str(pair[1])) > 5:
                countour_cleaned_vertices.append(pair)

        countour_polygon = PolygonArea(countour_cleaned_vertices)
        #print(countour_polygon.area)
        data_for_dataframe = []
        for key, value in voronoi_player_dict_defense_clean.items():
            voronoi_polygon = PolygonArea(value)
            intersection = countour_polygon.intersection(voronoi_polygon)

            overlap_area = intersection.area
            overlap_percentage = (overlap_area / countour_polygon.area) * 100  # Assuming percentage calculation

            # Update cumulative_dict
            cumulative_dict[key] = cumulative_dict.get(key, 0) + overlap_percentage

            # Prepare data for the DataFrame
            data_for_dataframe.append([game_id, play_id, frame_id, key, overlap_percentage])

    return data_for_dataframe

cumulative_overlap_dict = {}  # Initialize the dictionary to store cumulative data
all_data = []  # List to store all data rows for the DataFrame

# Loop through each game






# Create DataFrame from the collected data


In [ ]:
all_data = []  # Make sure this is initialized before the loop

# Loop through each game
# Assuming all_data is a list


for game_id in tracking['gameId'].unique():
    single_game = tracking[tracking['gameId'] == game_id]

    # Loop through each play in the game
    for play_id in single_game['playId'].unique():
        print(game_id, play_id)
        single_play = single_game[single_game['playId'] == play_id]
        frame_max = single_play['frameId'].max()
        frame_min = single_play['frameId'].min()

        # Loop through each frame in the play
        for frame_id in range(frame_min, frame_max+1):
            print(frame_min, frame_max, frame_id)
            try:
                # Try to calculate overlap for the frame
                frame_data = calculate_overlap_for_frame(frame_id, game_id, play_id, tracking, cumulative_overlap_dict)
                all_data.extend(frame_data)
            except Exception as e:
                # Log the error
                error_info = {
                    'game_id': game_id,
                    'play_id': play_id,
                    'frame_id': frame_id,
                    'error_message': str(e),
                    'type': 'error'  # This helps in identifying error logs later
                }
                all_data.append(error_info)
                print(f"Error processing frame {frame_id} in play {play_id}, game {game_id}: {e}")

            print('done')

# all_data now contains both successful frame data and error logs

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def process_frame(frame_id, game_id, play_id, tracking):
    try:
        frame_data = calculate_overlap_for_frame(frame_id, game_id, play_id, tracking, cumulative_overlap_dict)
        return frame_data
    except Exception as e:
        return {
            'game_id': game_id,
            'play_id': play_id,
            'frame_id': frame_id,
            'error_message': str(e),
            'type': 'error'
        }

# Assuming all_data is a list
all_data = []

# Adjust the number of workers based on your system's capabilities
num_workers = 4

with ThreadPoolExecutor(max_workers=num_workers) as executor:
    futures = []
    total_frames = 0

    for game_id in tracking['gameId'].unique():
        single_game = tracking[tracking['gameId'] == game_id]

        for play_id in single_game['playId'].unique():
            single_play = single_game[single_game['playId'] == play_id]
            frame_max = single_play['frameId'].max()
            frame_min = single_play['frameId'].min()

            for frame_id in range(frame_min, frame_max + 1):
                futures.append(executor.submit(process_frame, frame_id, game_id, play_id, tracking))
                total_frames += 1

    completed_frames = 0
    # Incremental collection of results with progress tracking
    for future in as_completed(futures):
        result = future.result()
        completed_frames += 1

        if isinstance(result, list):
            all_data.extend(result)
        else:
            all_data.append(result)

        print(f"Completed {completed_frames}/{total_frames} frames")

    print("All frames processed")

In [ ]:
column_names = ['gameId', 'playId', 'frameId', 'playerName', 'overlapPercentage']
overlap_df = pd.DataFrame(all_data, columns=column_names)

In [ ]:
overlap_df.head()

,gameId,playId,frameId,playerName,overlapPercentage
0,2022102310,56,3,Juan Thornhill,0.0
1,2022102310,56,3,Frank Clark,0.0
2,2022102310,56,3,Chris Jones,0.0
3,2022102310,56,3,Leo Chenal,0.0
4,2022102310,56,3,George Karlaftis,0.0
